In [ ]:
###########import packages##########
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import ensemble
from sklearn.tree import ExtraTreeRegressor
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from sklearn import ensemble
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score
from sklearn.model_selection import LeaveOneOut
seed=911
###########import packages##########
import catboost
import xgboost
import lightgbm
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import *
import pickle
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import  *
###########import packages##########
import tensorflow as tf
import keras
from keras import optimizers
from keras import regularizers
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.constraints import max_norm
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import Dropout 
from keras.models import Model
from keras.layers import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.wrappers.scikit_learn import KerasRegressor
from keras.constraints import maxnorm 
# from keras.wrappers import scikit_learn
from scikeras.wrappers import KerasClassifier, KerasRegressor
###########loading data##########
loo = LeaveOneOut()
# %matplotlib



In [ ]:
###########wrapping root mean square error for later calls##########
def compute_mae_mse_rmse(target,prediction):
    error = []
    for i in range(len(target)):
        error.append(target[i] - prediction[i])
    squaredError = []
    absError = []
    for val in error:
        squaredError.append(val * val)  # target-prediction之差平方
        absError.append(abs(val))  # 误差绝对值
    mae=sum(absError)/len(absError)  # 平均绝对误差MAE
    mse=sum(squaredError)/len(squaredError)  # 均方误差MSE
    RMSE=np.sqrt(sum(squaredError)/len(squaredError))
    R2=r2_score(target,prediction)
    return mae,mse,RMSE,R2
def gridsearch(model,param,algorithm_name,X_train,y_train,X_test,y_test):
    grid = GridSearchCV(model,param_grid=param,scoring='neg_mean_absolute_error',cv=10,n_jobs=8,verbose=-1)
    grid.fit(X_train,y_train)
    best_model=grid.best_estimator_
    ####Train####
    print("=========Train===========")
    prediction_train = best_model.predict(X_train)
    real_train=y_train.values
    prediction_train_series=pd.Series(prediction_train)
    real_train_series=pd.Series(real_train)
    corr_ann_train = round(prediction_train_series.corr(real_train_series), 5)
    error_val_train= compute_mae_mse_rmse(prediction_train,real_train)
    print(error_val_train)
    ####test####
    print("=========Test===========")
    prediction_test = best_model.predict(X_test)
    real_test=y_test.values
    prediction_test_series=pd.Series(prediction_test)
    real_test_series=pd.Series(real_test)
    corr_ann_test = round(prediction_test_series.corr(real_test_series), 5)
    error_val_test= compute_mae_mse_rmse(prediction_test,real_test)
    print(error_val_test)
    ####All####
    print("=========All===========")
    prediction_all = best_model.predict(data_input_full)
    real_all=data_output_full.values
    prediction_all_series=pd.Series(prediction_all)
    real_all_series=pd.Series(real_all)
    corr_ann_all = round(prediction_all_series.corr(real_all_series), 5)
    error_val_all= compute_mae_mse_rmse(prediction_all,real_all)
    print(error_val_all)
    
    print(algorithm_name)
    best_score=grid.best_score_
    print('Best Regressor:',grid.best_params_,'Best Score:', best_score)
    print('R2 TEST',error_val_test[3])
    fig=plt.figure()
    ax = fig.add_subplot(111)
    x_y_x=np.arange(-4,4,0.01)
    x_y_y=np.arange(-4,4,0.01)
    ax.scatter(prediction_train,real_train,c='blue',label='Train',alpha=0.25)
    ax.scatter(prediction_test,real_test,c='red',label='Test',alpha=0.75)
    ax.plot(x_y_x,x_y_y,c='black')
    plt.legend()
    plt.xlabel('Predicted_Overpotential@10 mA cm-2')
    plt.ylabel('Real_Overpotential@10 mA cm-2')
    return best_model,error_val_test[3]

In [ ]:
fl = open(r'./database_full_st.pkl','rb')
database_full=pickle.load(fl)

In [ ]:
data_input_full=database_full.iloc[:,0:55]
data_output_full=database_full.iloc[:,55]
X_train,X_test,y_train,y_test=train_test_split(data_input_full,data_output_full,test_size=0.1,random_state=seed)

In [ ]:
model_SVR = svm.SVR()
param_svr = {
'kernel':['linear', 'poly', 'rbf'],
'max_iter':[100,200,300,400,500,600,700,800,1000,1100,1200,1300,1400,1500],
'degree':[2,3,4],
'gamma':['scale','auto'],
'epsilon':[0.001,0.01,0.1,0.3,0.5,0.7,1],
'coef0':[100,200,300,400,500,600,700,800,1000,1100,1200,1300,1400,1500]
#     'coef0': [100], 'degree': [2], 'epsilon': [0.1], 'gamma': ['scale'], 'kernel': ['rbf'], 'max_iter': [400]
       }
SVR_full,SVR_full_score=gridsearch(model_SVR,param_svr,'Support Vector Regressor',X_train,y_train,X_test,y_test)

In [ ]:
model_KNeighborsRegressor = neighbors.KNeighborsRegressor()
param_knr = {
'n_neighbors':range(1,10),'weights':['uniform','distance'],
'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'],
'leaf_size':[2,10,20,30,40,50,100],
'p':range(1,10)
#     'algorithm': ['ball_tree'], 'leaf_size': [20], 'n_neighbors':[ 5], 'p': [4], 'weights': ['distance']
       }
KNR_full,KNR_full_score=gridsearch(model_KNeighborsRegressor,param_knr,'K Nearest Neighbor Regressor',X_train,y_train,X_test,y_test)

In [ ]:
model_LGBMRegressor=LGBMRegressor(random_state=1,verbose=0)
param_lgbm = {
'boosting_type':['gbdt','rf'],
'learning_rate':[0.001,0.002,0.004,0.005,0.006,0.008,0.01,0.02,0.04,0.06,0.05,0.06,0.08,0.1,0.12,0.14,0.15,0.16,0.18,0.2,0.4,0.5,0.6,0.8,1],
'subsample':[0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1],
'n_estimators':[50,100,200,400],
'max_depth':[5,7,9,11,13,-1],
'reg_alpha':[0,0.001,0.01,0.0001,0.00001],
'reg_lambda':[0,0.001,0.01,0.0001,0.00001]
#     'boosting_type':[ 'gbdt'], 'learning_rate': [0.12], 'max_depth': [7], 'n_estimators': [400], 'reg_alpha': [0.001], 'reg_lambda': [1e-05], 'subsample': [0.4]
}
LGBM_full,LGBM_full_score=gridsearch(model_LGBMRegressor,param_lgbm,'LightGBM',X_train,y_train,X_test,y_test)

In [ ]:
model_XGRegressor=XGBRegressor(random_state=1)
param_xg={
'booster':['gbtree'],
'learning_rate':[0.001,0.002,0.004,0.005,0.006,0.008,0.01,0.02,0.04,0.06,0.05,0.06,0.08,0.1,0.12,0.14,0.15,0.16,0.18,0.2,0.4,0.5,0.6,0.8,1],
'n_estimators':[100,200,400],
'max_depth':[3,5,7,9,11,13,-1],
'subsample':[0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1],
'reg_alpha':[0,0.001,0.01,0.0001,0.00001],
'reg_lambda':[0,0.001,0.01,0.0001,0.00001]
# 'booster': ['gbtree'], 'learning_rate': [0.05], 'max_depth': [11], 'n_estimators': [100], 'reg_alpha': [1e-05], 'reg_lambda': [0], 'subsample': [0.45 ]   
}
XG_full,XG_full_score=gridsearch(model_XGRegressor,param_xg,'XGBoost',X_train,y_train,X_test,y_test)

In [ ]:
model_CatRegressor=catboost.CatBoostRegressor(random_state=1,verbose=0)
param_cat = {
'learning_rate':[0.001,0.002,0.004,0.005,0.006,0.008,0.01,0.02,0.04,0.06,0.05,0.06,0.08,0.1,0.12,0.14,0.15,0.16,0.18,0.2],
'n_estimators':[100,200,400],
"boosting_type":["Plain"],
'max_depth':[5,7,9,11],
'subsample':[0.4,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1],
'reg_lambda':[0,0.001,0.01,0.0001,0.00001]
#     'boosting_type': ['Plain'], 'learning_rate': [0.08], 'max_depth': [7], 'n_estimators': [400], 'reg_lambda': [0], 'subsample': [0.8]
}
CAT_full,CAT_full_score=gridsearch(model_CatRegressor,param_cat,'CatBoost',X_train,y_train,X_test,y_test)

In [ ]:
model_GradientBoostingRegressor = ensemble.GradientBoostingRegressor(random_state=1)
###########defining the parameters dictionary##########
param_GB = {
'learning_rate':[0.001,0.002,0.004,0.005,0.006,0.008,0.01,0.02,0.04,0.06,0.05,0.06,0.08,0.1,0.12,0.14,0.15,0.16,0.18,0.2,0.4,0.5,0.6,0.8,1],
'n_estimators':[50,100,200,400],
'max_depth':[3,5,7,9,11,13,16],
'criterion':['friedman_mse','mae','mse'],
'max_features':['auto','sqrt','log2'],
'loss':['ls', 'lad', 'huber', 'quantile']
#     'criterion': ['friedman_mse'], 'learning_rate': [0.05], 'loss': ['lad'], 'max_depth': [9], 'max_features': ['auto'], 'n_estimators': [100]
}
GB_full,GB_full_score=gridsearch(model_GradientBoostingRegressor,param_GB,'GradientBoost',X_train,y_train,X_test,y_test)

In [ ]:
###########RandomForest gridsearch CV for best hyperparameter##########
model_RandomForestRegressor = ensemble.RandomForestRegressor(random_state=1)
###########defining the parameters dictionary##########
param_RF = {
'n_estimators':[50,100,200,400,None],
'max_depth':[3,5,7,9,11,None],
'criterion':['mse','mae'],
'max_features':['auto','sqrt','log2']
# 'criterion': ['mae'], 'max_depth': [11], 'max_features': ['auto'], 'n_estimators': [400]
}
RF_full,RF_full_score=gridsearch(model_RandomForestRegressor,param_RF,'Random Forest',X_train,y_train,X_test,y_test)

In [ ]:
model_DecisionTreeRegressor = tree.DecisionTreeRegressor(random_state=1)
param_dt={
'max_depth':[5,6,7,8,9,10,11,None],
'max_features':['auto','sqrt','log2'],
'criterion' : ["mse", "friedman_mse", "mae"],
'splitter' : [ "best",'random']
# 'criterion': ['mae'], 'max_depth': [6], 'max_features': ['auto'], 'splitter': ['best']
}
DT_full,DT_full_score=gridsearch(model_DecisionTreeRegressor,param_dt,'Decision Tree',X_train,y_train,X_test,y_test)

In [ ]:
model_AdaBoostRegressor = ensemble.AdaBoostRegressor(random_state=1)
param_ada={
'n_estimators':[50,100,200,400,800],
'learning_rate':[0.001,0.002,0.004,0.005,0.006,0.008,0.01,0.02,0.04,0.06,0.05,0.06,0.08,0.1,0.12,0.14,0.15,0.16,0.18,0.2,0.4,0.5,0.6,0.8,1],
'loss':['linear', 'square', 'exponential']  
# 'learning_rate': [0.02], 'loss': ['exponential'], 'n_estimators': [100]
}
ADA_full,ADA_full_score=gridsearch(model_AdaBoostRegressor,param_ada,'AdaBoost',X_train,y_train,X_test,y_test)

In [ ]:
def create_ANN_model_1layer(X,learning_rate,regular_term=0.001,neuron_number=50,drop_out_rate=0):
    regularizer=keras.regularizers.l2(regular_term)
    model = Sequential() 
    model.add(Dense(neuron_number, input_dim=X.shape[1], kernel_initializer='random_normal',
                    bias_initializer='random_normal',activation='relu',kernel_regularizer=regularizer)) 
    model.add(Dropout(drop_out_rate))
    model.add(Dense(neuron_number, input_dim=neuron_number, kernel_initializer='random_normal',
                    bias_initializer='random_normal',activation='relu',kernel_regularizer=regularizer)) 
    model.add(Dropout(drop_out_rate))
    model.add(Dense(1, input_dim=neuron_number, activation='linear'))
    adam=optimizers.Adam(learning_rate)
    model.compile(loss='mae')
    return model
def create_ANN_model_2layer(X,learning_rate,regular_term=0.001,neuron_number=50,drop_out_rate=0):
    regularizer=keras.regularizers.l2(regular_term)
    model = Sequential() 
    model.add(Dense(neuron_number, input_dim=X.shape[1], kernel_initializer='random_normal',
                    bias_initializer='random_normal',activation='relu',kernel_regularizer=regularizer)) 
    model.add(Dropout(drop_out_rate))
    model.add(Dense(neuron_number, input_dim=neuron_number, kernel_initializer='random_normal',
                    bias_initializer='random_normal',activation='relu',kernel_regularizer=regularizer)) 
    model.add(Dropout(drop_out_rate))
    model.add(Dense(neuron_number, input_dim=neuron_number, kernel_initializer='random_normal',
                    bias_initializer='random_normal',activation='relu',kernel_regularizer=regularizer)) 
    model.add(Dropout(drop_out_rate))
    model.add(Dense(1, input_dim=neuron_number, activation='linear'))
    adam=optimizers.Adam(learning_rate)
    model.compile(loss='mae')
    return model

In [ ]:
model_ANNRegressor1= KerasRegressor(build_fn=create_ANN_model_1layer(X=data_input_full,learning_rate=0.01), verbose=0)
model_ANNRegressor2= KerasRegressor(build_fn=create_ANN_model_2layer(X=data_input_full,learning_rate=0.01), verbose=0)

In [ ]:
epochs_list=[]
for i in range(10,210,10):
    epochs_list.append(i)

In [ ]:
# 设置参数候选值
batch_size_list = [8,16,32]
optimizers_list=['sgd', 'rmsprop', 'adam', 'adagrad']
param_ann = dict(batch_size=batch_size_list, 
                 epochs=epochs_list,
                optimizer=optimizers_list
                )

In [ ]:
# param_ann_1={'batch_size': [16], 'epochs': [200], 'optimizer': ['adagrad']}
# param_ann_2={'batch_size': [16], 'epochs': [150], 'optimizer': ['sgd']}

In [ ]:
ANN_1layer,ANN_1layer_score=gridsearch(model_ANNRegressor1,param_ann,'Artificial Neural Network',X_train,y_train,X_test,y_test)

In [ ]:
ANN_2layer,ANN_2layer_score=gridsearch(model_ANNRegressor2,param_ann,'Artificial Neural Network',X_train,y_train,X_test,y_test)

In [ ]:

def committee_predictor(models_with_scores, X_train, y_train, X_test, y_test, data_input_full, data_output_full):
    # Create the committee predictor
    def predict(X):
        weighted_predictions = np.zeros(X.shape[0])
        total_weight = 0
        for model, weight in models_with_scores:
            if weight < 0:
                weight = 0
            predictions = model.predict(X)
            weighted_predictions += predictions * weight
            total_weight += weight
        if total_weight == 0:
            return weighted_predictions
        return weighted_predictions / total_weight
    
    # Calculate error metrics
    def compute_errors(predictions, real):
        error = predictions - real
        squared_error = np.square(error)
        abs_error = np.abs(error)
        mae = np.mean(abs_error)
        mse = np.mean(squared_error)
        rmse = np.sqrt(mse)
        r2 = r2_score(real, predictions)
        return mae, mse, rmse, r2
    
    # Predict and compute errors for train, test, and all data
    def compute_all_errors():
        # Train
        predictions_train = predict(X_train)
        real_train = y_train.values
        error_val_train = compute_errors(predictions_train, real_train)
        
        # Test
        predictions_test = predict(X_test)
        real_test = y_test.values
        error_val_test = compute_errors(predictions_test, real_test)
        
        # All
        predictions_all = predict(data_input_full)
        real_all = data_output_full.values
        error_val_all = compute_errors(predictions_all, real_all)
        
        return error_val_train, error_val_test, error_val_all
    
    error_val_train, error_val_test, error_val_all = compute_all_errors()
    
    # Output results
    print("=========Train===========")
    print(error_val_train)
    print("=========Test===========")
    print(error_val_test)
    print("=========All===========")
    print(error_val_all)
    
    return error_val_train, error_val_test, error_val_all

# Example usage
models_with_scores = [   
    (ANN_2layer, ANN_2layer_score),
    (ANN_1layer, ANN_1layer_score),
    (ADA_full, ADA_full_score),
    (DT_full, DT_full_score),
    (RF_full, RF_full_score),
    (GB_full, GB_full_score),
    (CAT_full, CAT_full_score),
    (XG_full, XG_full_score),
    (LGBM_full, LGBM_full_score),
    (KNR_full, KNR_full_score),
    (SVR_full, SVR_full_score)
]
error_val_train, error_val_test, error_val_all = committee_predictor(models_with_scores, X_train, y_train, X_test, y_test, data_input_full, data_output_full)